# Как работает сборщик мусора

CPython (стандартный интерпретатор Python) использует два механизма для сборки мусора — подсчет ссылок на объекты и generational garbage collector (модуль gc в стандартной библиотеке Python).

### Алгоритм подсчета ссылок

В Python все переменные являются ссылками на объекты. Естественно, на один объект может ссылаться несколько переменных.


Так выглядит структура CPython, на основе которой реализованы другие, более сложные примитивы CPython. Здесь переменная **ob_refcnt** — переменная, которая увеличивается каждый раз, когда на данный объект создается ссылка.

Когда эта связь пропадает, естественно, счетчик ссылки объекта уменьшается.

Каждый раз, когда счетчик ссылок становится равным нулю, запускается механизм уничтожения объекта, при этом удаляются и ссылки, которые этот объект имел к другим объектам, таким образом, уничтожение одного объекта может повлечь волну удалений других объектов.

Основной проблемой, которой обладает алгоритм подсчета ссылок, является невозможность разрешения циклических зависимостей. Эту проблему призван решить generational garbage collector.

с помощью функции **sys.getrefcount()**, всегда можно узнать количество ссылок на объект.

In [1]:
import sys
test_list = [1,2,3]
print(sys.getrefcount(test_list))
test_list_2 = test_list
print(sys.getrefcount(test_list_2))

2
3


Функция **sys.getrefcount()** обычно возвращает количество ссылок большее на единицу, чем ожидалось.

Это связано с созданием временной ссылки на аргумент, который передается в функцию.

### Generational garbage collector (GC)

Циклическая ссылка — это когда один или несколько объектов ссылаются друг на друга.

In [2]:
a = []
a.append(a) #Здесь список ссылается сам на себя.

In [3]:
# Здесь словари ссылаются друг на друга.
a = {}
b = {}
a["b"] = b
b["a"] = a

Если вызвать метод del(), то произойдет удаление ссылок на объекты. Если бы не было GC, то объекты так бы и остались висеть в памяти, хотя и были бы недоступны для разработчика.

GC все объекты разделяет на три поколения. Изначально все объекты помещаются в первое поколение, живут там некоторое время и большинство из них очищается, остальная часть перемещается во второе поколение и потом в третье. Чем выше поколение, тем реже оно сканируется на мусор. 

Для выявления циклических ссылок GC итерирует каждый объект в поколении и временно удаляет все ссылки, на которые этот объект ссылается. После полного обхода, все обьекты, у которых счетчик ссылок меньше двух, считаются недоступными и удаляются.

GC как модуль предоставляет возможность управлять сборщиком мусора для циклических ссылок.

In [5]:
import gc
gc.enable() # включение сборщика мусора
gc.disable() # выключение сборщика мусора
gc.collect(generation=2) # явно инициирует проход сборщика мусора до его автоматического запуска

36

***Полезная ссылка***

[Более подробно о возможностях модуля gc можно прочитать тут][1]

[1]:https://docs.python.org/3/library/gc.html

# Менеджмент памяти

Если новый создаваемый объект занимает в памяти больше, чем 512 байт, то Python отправляет его на стандартный С-аллокатор.

Для оптимизации выделения памяти под небольшие объекты размером менее 512 байт, Python заранее выделяет большие блоки памяти, которые разделены на три уровня — арены, пулы, блоки.

**Блок** — это участок памяти определенного размера, каждый блок содержит только один Python-объект фиксированного размера, кратного 8. Размер блока от 8 до 512 байт.

**Пул** — это коллекция блоков одинакового размера. Обычно размер пула — это 4 КБ. Если объект уничтожается, то занимаемая им память не отдается операционной системе, а сохраняется для будущих объектов такого же размера.

Пул и блоки не выделяют память напрямую, а используют память, выделенную аренами

**Арена** — участок памяти в 256 КБ, выделенный из кучи операционной системы, которая вмещает в себя 64 пула.

Схематично арена выглядит так:

|        |Arena   |   |
|--------|------|--------|
| Pool(4kB)|Pool(4kB) | Pool(4kB)|
|Pool(4kB)|Pool(4kB)|Pool(4kB)|
|Pool(4kB)|FreePool(4kB)|FreePool(4kB)|
|...|...|...|


# Основы процессов и потоков

**Процесс** — это часть виртуальной памяти и ресурсов, которые операционная система выделяет для выполнения программы.

Каждый процесс выполняется в отдельном адресном пространстве и не может получить доступ к ресурсам другого процесса. 

**Поток** — это отдельная «нить» (поток) выполнения внутри процесса. 

У каждой программы есть как минимум один процесс, а у каждого процесса — минимум один поток, который называют главным. 

Разница между потоками и процессами состоит в том, что потоки используют память, выделенную под процесс, процессы в свою очередь обмениваются данными между собой только с помощью механизмов межпроцессного взаимодействия, а потоки обращаются к ресурсам друг друга напрямую. 

**Многопоточность** — это разбиение процесса программы на множество потоков, которые работают параллельно друг с другом. Например, одновременное получение нескольких html-страниц с разных сайтов.

**Многопроцессорность** — это разбиение программы на несколько независимых процессов. Например, один процесс в программе реализует основную бизнес-логику, другой процесс запускает веб-сервер, необходимый для мониторинга жизни приложения.

 *в Python и его стандартном интерпретаторе CPython только кажется, что потоки выполняются параллельно, на самом деле они выполняются последовательно. Это связано с GIL (Global Interpreter Lock), который ограничивает Python на один запущенный поток в единицу времени.*
 
 **GIL — это блокировка самого интерпретатора Python. То есть, она является единственной блокировкой в системе и позволяет решить проблему взаимоблокировок, но в свою очередь делает все приложения однопоточными.**

#### Так хорошо ли GIL или плохо?
**В не зависимости от количества ядер процессора, любая многопоточная программа на Python не сможет раскрыть потенциал и будет работать даже медленнее однопоточной за счет переключения GIL между потоками.**

Если разделить все программы на CPU-зависимые (обработка изображений, умножение матриц) и I/O-зависимые (связь по сети, обращение к БД), то можно понять, что использование потоков и GIL не несет ничего критического при I/O-операциях, т.к. время, затраченное Python на переключение потоков, будет компенсировано временем I/O-операций. 

Модуль **threading** отвечает за создание и работу с потоками

In [3]:
import threading
import time
def thread_function(name):
    print(name, " - thread starting")
    time.sleep(3)
    print(name, " - after sleep")

if __name__ == "__main__":
    print("before create Thread")
    x = threading.Thread(target=thread_function, args=(1,))
    print("before running Thread")
    x.start()
    print("Wait thread finish")
    #x.join() # указание основному потоку дождаться завершения потока x.
    print("all done")

before create Thread
before running Thread
1  - thread starting
Wait thread finish
all done
1  - after sleep


В данном примере:

1. Мы импортируем модуль **threading.**
2. Создаем объект класса **Thread**, передавая ему на вход функцию, с которой он начнет работу, и аргументы для этой функции. 
3. Методом **start()** можно запустить поток, и, когда он завершит выполнение функции **thread_function()**, он автоматически завершится.

Также во время создания объекта класса **Thread** можно передать параметр **daemon=True**, который позволит создать daemon-поток.

x.join() — это указание основному потоку дождаться завершения потока x. Это может быть полезно в случае, когда дочерние потоки делают какую-то работу, а основной поток впоследствии работает с данными, которые подготовили дочерние потоки.

In [2]:
x = threading.Thread(target=thread_function, args=(1,), daemon=True)

В теории daemon-процесс — это процесс, который работает в фоновом режиме.

для запуска нескольких потоков можно комбинировать потоки, помещая в список

In [8]:
import threading
import time
def thread_function(name):
    print(name, " - thread start")
    time.sleep(2)
    print(name, "- thread job's done")
if __name__ == "__main__":
    threads = []
    for index in range(3):
        print("create thread - ", index)
        x = threading.Thread(target=thread_function, args=(index,))
        threads.append(x)
        x.start()

    for index, thread in enumerate(threads):
        print("before join - ", index)
        thread.join()
        print("after join - ", index)

create thread -  0
0  - thread start
create thread -  1
1  - thread start
create thread -  2
2  - thread start
before join -  0
0 - thread job's done
after join -  0
before join -  1
1 - thread job's done
after join -  1
before join -  2
2 - thread job's done
after join -  2


Помимо создания нескольких потоков и хранения их в списке, в Python есть возможность использовать ThreadPoolExecutor, который позволяет создать N потоков более просто.

In [12]:
from concurrent.futures import ThreadPoolExecutor
import threading
import time
def thread_function():
    time.sleep(2)
    print("thread_function Executed {}".format(threading.current_thread()))
def main():
    executor = ThreadPoolExecutor(max_workers=3)
    task1 = executor.submit(thread_function)
    task2 = executor.submit(thread_function)
if __name__ == '__main__':
    main()

thread_function Executed <Thread(ThreadPoolExecutor-3_1, started 24448)>
thread_function Executed <Thread(ThreadPoolExecutor-3_0, started 26684)>


В этом примере создается ThreadPoolExecutor, с количеством потоков равным 3, и с помощью объекта executor передается функция, которую нужно выполнить. Как видно из вывода, разные потоки выполняют эту функцию.

# Примитивы синхронизации потоков

In [ ]:
class DataBase:
    def __init__(self):
        self.value = 0
    def update(self, name):
        print(name, " - start thread")
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        print(name, " - finish thread")

**Lock** — это блокировка, которая может одновременно удерживаться только одним потоком.

In [13]:
lock = threading.Lock()
lock.acquire() # Выполнит блокировку данного участка кода
# какой-то код
lock.release() # освобождение блокировки

 можно использовать контекстный менеджер и не беспокоиться о необходимости выполнять явную блокировку / разблокировку.

**Семафор** (связка из блокировки и счётчика потоков) чем-то похож на Lock с той лишь разницей, что в него встроен счетчик, который блокирует доступ в случае превышения числа потоков, которые удерживают семафор. 

In [ ]:
import threading
max_connections = 10
semaphore = threading.BoundedSemaphore(max_connections)
semaphore.acquire() # уменьшает счетчик (-1)
... доступ к общим ресурсам
semaphore.release() # увеличивает счетчик (+1)

С каждым acquire() счетчик уменьшается, с release() — увеличивается, но когда счетчик равен 0, новый поток будет вынужден ждать, пока не освободится место для него.

Так же во время разработки программы с многопоточностью удобно использовать модуль Queue, который реализует механизм очередей с поддержкой threadsafe. Это означает, что, используя очередь, можно безопасно обмениваться информацией между потоками.

In [19]:
import threading
import time
from queue import Queue
from threading import Thread
num_worker_threads=2
def do_work(item):
    time.sleep(1)
    print("my task is", item, "i am ", threading.current_thread())

def worker():
    while True:        
        item = q.get() # получаем задание из 
        print("get task - ", item)
        do_work(item) # выполняем работу
        q.task_done() # сообщаем о завершении работы
q = Queue()
for i in range(num_worker_threads): # Создаем и запускаем потоки
    t = Thread(target=worker)
    t.setDaemon(True)
    t.start()
for item in range(0, 5): # помещаем задания в очередь
    q.put(item)
q.join() # Ждем, пока не будут выполнены все задания

get task -  0
get task -  1
my task is 1 i am  <Thread(Thread-34, started daemon 9080)>
get task -  2
my task is 0 i am  <Thread(Thread-35, started daemon 13812)>
get task -  3
my task is 2 i am  <Thread(Thread-34, started daemon 9080)>
get task -  4
my task is 3 i am  <Thread(Thread-35, started daemon 13812)>
my task is 4 i am  <Thread(Thread-34, started daemon 9080)>


апускаем несколько потоков, создаем очередь и помещаем в нее задания. Потоки, используя безопасный метод q.get(), получают задания и выполняют их.

# Модуль multiprocessing

Модуль multiprocessing несет в себе возможность создавать процессы таким же образом, как и потоки из модуля threading. Таким образом, можно обойти GIL и получить настоящую параллельную работу. 

пример

In [69]:
import os
from multiprocessing import Process, current_process
def foo(number):
    proc_name = current_process().name
    return '{0} {1}'.format(number, proc_name)
    
if __name__ == '__main__':
    random_numbers = [5, 10, 15, 20, 25]
    process_list = []
    proc = Process(target=foo, args=(5,))
    for index, number in enumerate(random_numbers):
        proc = Process(target=foo, args=(number,))
        process_list.append(proc)
        proc.start()   
    proc = Process(target=foo, name='Test', args=(2,))
    proc.start()
    process_list.append(proc)
    for proc in process_list:
        proc.join()

In [68]:
process_list

[<Process name='Process-160' pid=23604 parent=14472 stopped exitcode=1>,
 <Process name='Process-161' pid=27224 parent=14472 stopped exitcode=1>,
 <Process name='Process-162' pid=27972 parent=14472 stopped exitcode=1>,
 <Process name='Process-163' pid=1568 parent=14472 stopped exitcode=1>,
 <Process name='Process-164' pid=28272 parent=14472 stopped exitcode=1>,
 <Process name='Test' pid=27336 parent=14472 stopped exitcode=1>]

Класс Process в качестве аргументов принимает:

Target — функцию, которая будет выполняться при запуске процесса.
Name — имя процесса, доступного через функцию current_process().
args — аргументы для функции target().

процессы поддерживают Lock для блокировки доступа к ресурсам.

In [72]:
from multiprocessing import Process, Lock, current_process
def print_function(item, lock):
    lock.acquire()
    try:
        print(item, current_process())
    finally:
        lock.release()
if __name__ == '__main__':
    lock = Lock()
    items = ['test1', 'test2', "test3"]    
    for item in items:
        p = Process(target=print_function, args=(item, lock))
        p.start()


<Process name='Process-176' pid=15516 parent=14472 started>
<Process name='Process-177' pid=14532 parent=14472 started>
<Process name='Process-178' pid=25328 parent=14472 started>


По примеру можем видеть, что, благодаря Lock, процессы работают с функцией по очереди.

Аналогом ThreadPoolExecutor является класс Pool, который позволяет запустить несколько процессов одновременно.

In [ ]:
from multiprocessing import Pool
def calc(number):
    return number * 2

if __name__ == '__main__':
    numbers = [5, 10, 20]
    pool = Pool(processes=3)
    print(pool.map(calc, numbers))

Мы создаем экземпляр класса Pool, указываем в processes, что хотим создать 3 процесса, а затем с помощью pool.map() передаем функцию для исполнения и список numbers, где впоследствии каждый из элементов списка будет подан на вход функции doubler().

Для связи между процессами можно использовать класс Queue, который так же реализован в модуле multiprocessing

In [ ]:
from multiprocessing import Process, Queue
stop_number = -1 
def task_creator(data, q):
    for item in data:
        q.put(item)
def consmuer(q):
    while True:
        data = q.get()
        print('data: {}'.format(data))
        processed = data * 2
        print(processed)
        if data is stop_number:
            break
if __name__ == '__main__':
    q = Queue()
    data = [5, 10, 13, -1]   
    process_one = Process(target=task_creator, args=(data, q))
    process_two = Process(target=consmuer, args=(q,))
    process_one.start()
    process_two.start()
    q.close()
    q.join_thread()
    process_one.join()
    process_two.join()

Здесь мы создаем два процесса и очередь. Один процесс при старте кладет данные в очередь, а другой считывает ее и выводит на экран. 

Очереди удобны, когда нужно связать между собой несколько процессов, например, одни кладут в очередь, другие обрабатывают.

Но в модуле multiprocessing так же есть класс Pipe, который позволяет связать между собой только два процесса.

In [ ]:
import multiprocessing  
def sender(conn, msgs):
    for msg in msgs:
        conn.send(msg)
        print("Sent the message: {}".format(msg))
    conn.close()

def receiver(conn):
    while 1:
        msg = conn.recv()
        if msg == "END":
            break
        print("Received the message: {}".format(msg))

if __name__ == "__main__":
    msgs = ["START", "END"]
    parent_conn, child_conn = multiprocessing.Pipe()
    p1 = multiprocessing.Process(target=sender, args=(parent_conn,msgs))
    p2 = multiprocessing.Process(target=receiver, args=(child_conn,))  
    p1.start()
    p2.start() 
    p1.join()
    p2.join()

В этом примере мы создаем два процесса, связываем их с помощью Pipe и организуем простую передачу текстовых сообщений от одного к другому.

Pipe возвращает два объекта — parent_conn и child_conn.

parent_conn — объект, который отправляет данные через Pipe.

child_conn — объект, которые принимает данные из Pipe.

Pipe полезно использовать, когда один процесс работает в фоне, например, мониторит доступность сети, а другой процесс хочет сходить по какому-нибудь адресу и спрашивает о доступности сети у другого процесса, а потом принимает на основе этого решение.


**Задание 1**

Для тренировки навыка многопоточного программирования предлагаем решить классическую задачу об обедающих философах.

Пять безмолвных философов сидят вокруг круглого стола, перед каждым философом стоит тарелка спагетти. Вилки лежат на столе между каждой парой ближайших философов.

Каждый философ может либо есть, либо размышлять. Прием пищи не ограничен количеством оставшихся спагетти — подразумевается бесконечный запас. Тем не менее, философ может есть только тогда, когда держит две вилки — взятую справа и слева (альтернативная формулировка проблемы подразумевает миски с рисом и палочки для еды вместо тарелок со спагетти и вилок).

Каждый философ может взять ближайшую вилку (если она доступна) или положить — если он уже держит её. Взятие каждой вилки и возвращение ее на стол являются раздельными действиями, которые должны выполняться одно за другим.

Вопрос задачи заключается в том, чтобы разработать модель поведения (параллельный алгоритм), при котором ни один из философов не будет голодать, то есть будет вечно чередовать приём пищи и размышления.

In [4]:
import threading
import time
import random

class Philosopher(threading.Thread):
    running = True
    def __init__(self, name, left_fork, right_fork):
        threading.Thread.__init__(self)
        self.name = name
        self.left_fork = left_fork
        self.right_fork = right_fork
    def run(self):
        while self.running ==True:
            time.sleep(random.uniform(5,10))
            print(f"{self.name} is hungry")
            self.dine()
    def dine(self):
        while self.running:
            self.left_fork.acquire()
            if not self.right_fork.locked():
                self.right_fork.acquire()
                break
            else:
                self.left_fork.release()
        else:
            return
        self.start_dining()
        self.left_fork.release()
        print(f'{self.name} stop eating, start thinking')

        
    def start_dining(self):
        print(f'{self.name} start eating')
        time.sleep(random.uniform(10,20))

def start():
    forks = [threading.Lock() for n in range(5)] 
    philosoohes_name = ['Sokrat','Niche', 'Aristotel','Kant','Mark']

    philosophers = [Philosopher(philosoohes_name[i], forks[i%5], forks[(i+1)%5]) for i in  range(5)]
    Philosopher.running = True
    
    for p in philosophers:
        p.start()
    time.sleep(50)
    Philosopher.running = False
    print('finish')
    
start()

Mark is hungry
Mark start eating
Niche is hungry
Niche start eating
Kant is hungry
Sokrat is hungryAristotel is hungry

Mark stop eating, start thinkingKant start eating

Niche stop eating, start thinking
Mark is hungry
Niche is hungry
Kant stop eating, start thinking
Kant is hungry
finish
